In [1]:
import pandas as pd
import numpy as np
import hdf5_getters as hd
import glob
import sklearn
from sklearn import preprocessing

In [2]:
#To see if all the 10,000 files are available for data extraction
h5_dir = "MillionSongSubset" #<file_path where the song_dataset is stored>

# Get a list of all .h5 files in the directory and its subdirectories
h5_files = glob.glob(h5_dir + '/**/*.h5', recursive=True)
len(h5_files) #should be 10,000

10000

In [3]:
#Reading the dataset
df = pd.read_csv("3593_song_newfeatures_genre_dataset.csv")

In [4]:
#Filtering out data for International genre 
df_song = df[df['genre'] == 'International']
df_song

,Unnamed: 0,id,spectral_centroid_mean,spectral_centroid_dev,rolloff_point_mean,rolloff_point_dev,flux_mean,flux_dev,variability_mean,variability_dev,compactness_mean,compactness_dev,rms_mean,rms_dev,low_energy_window_mean,low_energy_window_dev,zero_crossing_mean,zero_crossing_dev,title,genre
31,31,TRBGFXJ12903CAC91E,13.970,10.9400,0.09198,0.071730,0.000943,0.001483,0.002678,0.001643,1612.0,167.2,0.10500,0.06038,0.5658,0.06637,36.220,24.360,b'Min Xechnas (Forse)',International
37,37,TRBGZED128F425DF76,1.824,0.8224,0.01071,0.005944,0.000685,0.001163,0.001916,0.001193,1595.0,230.9,0.07036,0.03916,0.5967,0.03854,4.629,3.434,b'Primal Forces',International
38,38,TRBGICJ128F427BA9A,8.538,12.6700,0.05356,0.093300,0.001374,0.002604,0.003275,0.001970,1603.0,216.8,0.12420,0.06757,0.5651,0.08159,26.580,28.360,b'Beni Aglatma',International
90,90,TRBGKLM128F425CF97,6.442,4.1710,0.03678,0.034180,0.005027,0.009449,0.006632,0.003316,1703.0,242.0,0.25200,0.11250,0.5134,0.06430,16.840,10.880,b'Phrase',International
132,132,TRBHQBT128F932AFAD,10.580,6.9090,0.07054,0.056760,0.004231,0.006805,0.005253,0.003167,1624.0,206.7,0.20300,0.11510,0.5989,0.05542,29.490,17.720,b'Winds of Change',International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,3531,TRAYNGT128F4262F7C,8.359,8.1130,0.05081,0.072610,0.001756,0.003041,0.003894,0.002009,1673.0,189.7,0.14880,0.07109,0.5646,0.05983,28.440,22.560,b'Lefkes Selides',International
3537,3537,TRAYPZB12903CE763B,11.940,5.1320,0.07437,0.040720,0.000146,0.000259,0.001752,0.000775,1957.0,274.2,0.06842,0.02833,0.5710,0.07154,31.280,14.550,b'Nordfjorloftet',International
3556,3556,TRAYJCD128F42802B4,7.827,7.4460,0.05611,0.078870,0.004787,0.006183,0.006377,0.002701,1600.0,192.0,0.24440,0.09163,0.5247,0.05079,26.910,21.140,b'La Pr\xc3\xa9tention De Rien',International
3566,3566,TRAYSTC128F92F526B,7.274,8.3390,0.04267,0.054410,0.000233,0.000287,0.001948,0.000989,1633.0,263.7,0.07467,0.03630,0.5450,0.09507,19.240,18.420,b'The Anniversary Waltz',International


In [5]:
#Creating lists of Genre and Track_id
track_id = df_song['id'].tolist()
genre = df_song['genre'].tolist()

In [6]:
genre

['International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'International',
 'Internat

In [7]:
#Function to extract the cleaned track_id
def clean(text):
    text = str(text)
    text = text.replace("b'", "")
    text = text[:-1]
    return text

In [8]:
#Collecting segment info for Loudness only for those songs in the 3953_dataset
batch_size = 10

song_id = []
loud_max_seg = []
pitch_seg = []
timbre_seg = []

for i in range(0, len(h5_files), batch_size):
    # Get the batch of files
    batch_files = h5_files[i:i+batch_size]
    
    # Loop through the files in the batch
    for file_name in batch_files:
        h5 = hd.open_h5_file_read(file_name)
        
        id = hd.get_track_id(h5)
        id = clean(id)

        if id in track_id:
            #print("Collecting data")
            song_id.append(hd.get_track_id(h5))
            loud_max_seg.append(hd.get_segments_loudness_max(h5))
            timbre_seg.append(hd.get_segments_timbre(h5))
            pitch_seg.append(hd.get_segments_pitches(h5))
        
        h5.close()

In [9]:
#All are equal to the number of songs filtered out from the International genre
len(song_id), len(loud_max_seg), len(timbre_seg), len(pitch_seg), len(genre)

(147, 147, 147, 147, 147)

In [10]:
#Inspecting values of loudness for different segments for 2 songs.
loud_max_seg[0:2]

[array([-60.   , -36.145, -37.986, ...,  -6.528,  -5.615,  -6.379]),
 array([-60.   , -15.184, -15.391, ..., -24.391, -22.535, -23.647])]

In [11]:
#inspecting the segments' shape for all songs
for arr in loud_max_seg:
    print(arr.shape)

(1002,)
(1444,)
(958,)
(890,)
(760,)
(1448,)
(578,)
(595,)
(2019,)
(883,)
(4046,)
(857,)
(748,)
(2240,)
(1988,)
(1000,)
(191,)
(1577,)
(1469,)
(1380,)
(771,)
(1836,)
(674,)
(15,)
(746,)
(924,)
(654,)
(2015,)
(1553,)
(691,)
(955,)
(1084,)
(2982,)
(1261,)
(696,)
(1150,)
(575,)
(598,)
(1113,)
(1096,)
(833,)
(1108,)
(965,)
(1075,)
(648,)
(515,)
(870,)
(988,)
(1181,)
(1164,)
(2531,)
(1008,)
(1009,)
(1574,)
(1511,)
(1453,)
(996,)
(795,)
(1218,)
(1432,)
(869,)
(1078,)
(1428,)
(554,)
(2145,)
(870,)
(1030,)
(1894,)
(862,)
(1278,)
(822,)
(626,)
(582,)
(991,)
(742,)
(176,)
(1305,)
(443,)
(542,)
(1030,)
(689,)
(211,)
(1285,)
(1180,)
(583,)
(1881,)
(977,)
(809,)
(1084,)
(870,)
(1178,)
(1554,)
(983,)
(613,)
(1068,)
(509,)
(960,)
(818,)
(1013,)
(790,)
(1399,)
(700,)
(874,)
(857,)
(728,)
(892,)
(720,)
(1513,)
(813,)
(737,)
(533,)
(1739,)
(681,)
(547,)
(2130,)
(681,)
(705,)
(677,)
(478,)
(750,)
(1499,)
(997,)
(717,)
(2043,)
(686,)
(735,)
(940,)
(731,)
(1196,)
(1001,)
(430,)
(1716,)
(481,)
(267,)
(670,)

In [12]:
#Installing Librosa Library for creating spectrogram
!pip install librosa
import librosa
import librosa.display
import numpy as np

In [13]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Padding the loudness arrays to be of the same length 
max_length = max(len(arr) for arr in loud_max_seg)
padded_loud_max_seg = []
for arr in loud_max_seg:
    padded_arr = np.pad(arr, (0, max_length - len(arr)), mode='constant')
    padded_loud_max_seg.append(padded_arr)
concatenated_loudness = np.vstack(padded_loud_max_seg)

# Standardization (Z-score normalization)
mean_val = np.mean(concatenated_loudness)
std_val = np.std(concatenated_loudness)
scaled_loudness = (concatenated_loudness - mean_val) / std_val

sr= 22050

# Computing the spectrogram using librosa
spec = librosa.feature.melspectrogram(y=None, sr=sr, S=scaled_loudness, n_fft=2048, hop_length=512, n_mels=64)

# Converting the power spectrogram to dB scale
spec_db = librosa.power_to_db(spec, ref=np.max)

# Plotting the spectrogram
plt.figure(figsize=(10, 5))
librosa.display.specshow(spec_db, x_axis='time', y_axis='mel', sr=sr, hop_length=512, cmap='coolwarm')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram for International genre')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()